In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from waveletUtil import *
from ipywidgets import VBox, IntSlider
from skimage.util import view_as_blocks
from skimage.util import montage
from skimage.transform import rescale

In [2]:
I = plt.imread('b.png')
I = I[...,:3].copy()

In [3]:
from skimage.transform import rescale
plt.figure()
plt.imshow(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
s = 8

In [5]:
H = makeDCTMatrix(s)

In [34]:
# The coordinates
xs = np.meshgrid(np.arange(s), np.arange(s), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)

In [35]:
Ib = view_as_blocks(I, block_shape = (8,8,3))

In [36]:
Ib.shape

(100, 100, 1, 8, 8, 3)

In [49]:
def generateCoef(sI, H):
    '''
    Generate coefficients for one block, a block of shape 8x8
    The CT gonna have the shape 8*8*3 
    Unordered
    '''
    CT = np.zeros(sI.shape)
    for chan in range(3):
        CT[...,chan] = np.matmul(H, np.matmul(sI[...,chan], H.transpose()))
    mags = CT[...,0].ravel(order='F')
    darg = np.argsort(mags)
    darg = list(reversed(darg))
    return CT, darg

In [50]:
# Sort by mag, we need only 1 channel to be honest
# mags = CT.ravel(order='F')
# darg = np.argsort(mags)
# darg.reverse()

In [51]:
xs1 = np.meshgrid(np.arange(Ib.shape[0]), np.arange(Ib.shape[1]), indexing='ij')
coords1 = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs1]], axis=1)

### WHAT I NEED TO DO
- Generate each CT and sort by magnitude
- Figure out how to do the coefficient and basis correctly?

In [52]:
# this is inefficient because you are basically doing mul in 64, keep track of what u have, add one
def make_image(change, H, sI):
    global coords
    CT, darg = generateCoef(sI, H)
    ReI = np.zeros(sI.shape)
    Bij = np.outer(H[0, :], H[0, :])
    for ind in range(change):
        i,j = coords[darg[ind]] #coords[darg[x]] if x in range(len(darg))

    #Construct that Haar basis and display it
        Bij = np.outer(H[i, :], H[j, :])
    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
        for chan in range(3):
            ReI[...,chan] += CT[i,j,chan]*Bij
    return Bij, ReI

In [53]:
def mergeImg(change, Ib, H):
    global coords1
    Ic = np.squeeze(Ib)

    blockView = Ic.reshape([Ic.shape[0]*Ic.shape[1]] + list(Ic.shape[2:]))
    newImageBlocks = np.zeros(blockView.shape)
    
    for ind, block in enumerate(blockView):
        i, j = coords1[ind]
        base, bT = make_image(change, H, Ib[i,j,0]) # Some transform of the block
    # bT is a (3,) array of the average color of the block
    # This line sets each of the 8x8 pixels to be the (1,1,3) version of the bT
        newImageBlocks[ind][:] = bT
    I_new = montage(newImageBlocks[:,:], grid_shape=[Ib.shape[0], Ib.shape[1]], multichannel=True)
    return I_new

In [56]:
init = mergeImg(1, Ib, H)
plt.ioff()
plt.clf()

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#FloatLogSlider
slider = IntSlider(
    orientation='horizontal',
    value=1,
    min=1,
    max=64,
    step=1,
    description='Base element'
)

fig_args = {'num':' ', 'frameon':True}
fig, ax = plt.subplots(1,2, figsize=(10,3), **fig_args) 

# display artists I'll update
# adisp = ax[0].imshow(basis, cmap='gray', vmin=-0.5, vmax=0.5)
hdisp = ax[0].imshow(init, cmap='magma', vmin=0, vmax=1)
ax[1].imshow(I)
ax[1].set_title("OG")

def update_image(change):
    global H, coords
    i, j = coords[change.new-1]
#     new_base = np.outer(H[i, :], H[j, :])
    new_img = mergeImg(change.new, Ib, H)
    hdisp.set_array(new_img)
    # need to reset the color limits each time since gaus_h is changing a lot. 
    hdisp.set_clim(0,1) 
    
#     adisp.set_array(new_base)
    
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_image, names='value')

VBox([slider, fig.canvas])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
